In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.signal import stft
import os
import pandas as pd
import sys
import glob
import torch
from torch import nn, Tensor
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import math
from sklearn.preprocessing import MinMaxScaler
import nbimporter
from sklearn.metrics import mean_squared_error, mean_absolute_error,mean_absolute_percentage_error,r2_score
from torch.nn.utils.rnn import pad_sequence
from tokenizers import Tokenizer, models, pre_tokenizers, trainers, decoders
from torch.optim.lr_scheduler import CosineAnnealingLR
import optuna
import json

In [2]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[batch_size, seq_len, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class Transformer(nn.Module):
    def __init__(self, vocab_size, max_seq_len, d_model, nhead, num_layers, output_dim, device, dropout_p,hidden_layers):
        super(Transformer, self).__init__()
        self.vocab_size = vocab_size
        self.max_seq_len = max_seq_len
        self.device = device
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout_p, max_seq_len)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model, nhead),
            num_layers=num_layers,
        )
        

        layers = []
        in_dim = d_model
        for out_dim in hidden_layers:
            layers.extend([
                nn.Linear(in_dim, out_dim),
                nn.Dropout(dropout_p),
                nn.LeakyReLU()
            ])
            in_dim = out_dim
        layers.append(nn.Linear(in_dim, output_dim))
        self.regressor = nn.Sequential(*layers)
 
 
    def forward(self, x):
        x = self.embedding(x)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1) 
        x = self.regressor(x)
        return x
    
class Trainer:
    def __init__(self, model, train_loader, valid_loader, num_epochs, patience, device, mse_loss, optimizer):
        self.model = model
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.optimizer = optimizer
        self.num_epochs = num_epochs
        self.patience = patience
        self.device = device
        self.mse_loss = mse_loss
        self.optimizer = optimizer
        self.train_losses = []
        self.valid_losses = []
        self.consecutive_greater_count = 0
        self.min_valid_loss = float('inf')
        
   # self.mse_loss = mse_loss
    # Training loop
    def train(self):
        self.consecutive_greater_count = 0 
        self.min_valid_loss = float('inf')
        for epoch in range(self.num_epochs):
            self.model.train()
            train_loss = 0
            for inputs, targets in self.train_loader:
            
                inputs, targets = inputs.to(self.device), targets.to(self.device)
            
                outputs = self.model(inputs)
                
                loss = self.mse_loss(outputs, targets)  #self.mse_loss

               
                self.optimizer.zero_grad()
                loss.backward()

                self.optimizer.step()

                train_loss += loss.item()

            train_loss /= len(self.train_loader)

            # Validation
            self.model.eval()
            valid_loss = 0

            with torch.no_grad():
                for inputs, targets in self.valid_loader:
                    inputs, targets = inputs.to(self.device), targets.to(self.device)

                    
                    outputs = self.model(inputs)

                    
                    loss = self.mse_loss(outputs, targets) #self.mse_loss

                    valid_loss += loss.item()

                valid_loss /= len(self.valid_loader)

            if math.isnan(valid_loss):
                print("Validation loss is NaN. Stopping training for this trial.")
                self.valid_losses.append(float('inf'))
                break
            
            #print(f'Epoch [{epoch+1}/{self.num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}')
            self.train_losses.append(train_loss)
            self.valid_losses.append(valid_loss)
            
            # Early stopping
            if valid_loss < self.min_valid_loss:
                self.min_valid_loss = valid_loss
                self.consecutive_greater_count = 0
                
            elif valid_loss == self.min_valid_loss:  # Check if the validation loss is the same as the minimum
                self.consecutive_greater_count += 1
                if self.consecutive_greater_count >= self.patience:
                    print(f'Stopping training after {epoch+1} epochs, validation loss not improving')
                    break

            else:
                self.consecutive_greater_count += 1
            if self.consecutive_greater_count == self.patience:
                # Save the best model 
                #torch.save(model.state_dict(), 'best_model.pth')
                print(f'Early stopping after {epoch+1} epochs')
                break
            #self.scheduler.step()
        return self.train_losses, self.valid_losses, self.model


In [3]:
class WordPiece:
    def __init__(self, vocab_size, max_seq_len):
        self.vocab_size = vocab_size
        self.max_seq_len = max_seq_len
        self.tokenizer = Tokenizer(models.WordPiece())
        self.tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
        self.tokenizer.decoder = decoders.WordPiece()

    def train_tokenizer(self, data):
        data_as_strings = [' '.join(sequence.astype(str)) for sequence in data]
        trainer = trainers.WordPieceTrainer(vocab_size=self.vocab_size, special_tokens=["[PAD]", "[UNK]"])
        self.tokenizer.train_from_iterator(data_as_strings, trainer=trainer)

    def tokenize_and_pad(self, data):
        data_as_strings = [' '.join(sequence.astype(str)) for sequence in data]
        tokenized_data = [self.tokenizer.encode(sequence).ids for sequence in data_as_strings]
        padded_data = pad_sequence(
            [torch.tensor(seq) for seq in tokenized_data],
            batch_first=True,
            padding_value=self.tokenizer.token_to_id("[PAD]")
        ).numpy()
        return padded_data

In [ ]:
class BPEtokenizer:
    def __init__(self, vocab_size, max_seq_len):
        self.vocab_size = vocab_size
        self.max_seq_len = max_seq_len
        self.tokenizer = Tokenizer(models.BPE())
        self.tokenizer.pre_tokenizer = pre_tokenizers.Whitespace()
        self.tokenizer.decoder = decoders.BPEDecoder()

    def train_tokenizer(self, data):
        data_as_strings = [' '.join(sequence.astype(str)) for sequence in data]
        trainer = trainers.BpeTrainer(vocab_size=self.vocab_size, special_tokens=["[PAD]", "[UNK]"])
        self.tokenizer.train_from_iterator(data_as_strings, trainer=trainer)

    def tokenize_and_pad(self, data):
        data_as_strings = [' '.join(sequence.astype(str)) for sequence in data]
        tokenized_data = [self.tokenizer.encode(sequence).ids for sequence in data_as_strings]
        padded_data = pad_sequence(
            [torch.tensor(seq) for seq in tokenized_data],
            batch_first=True,
            padding_value=self.tokenizer.token_to_id("[PAD]")
        ).numpy()
        return padded_data

In [4]:
from scipy.stats import pearsonr
class Evaluator:
    def __init__(self, y_true, y_pred):
        self.y_true = y_true
        self.y_pred = y_pred

    def normalized_mean_squared_error(self):
        err_pmus = np.array(self.y_true)
        err_pmus_hat = np.array(self.y_pred)
        nrmse = np.sqrt(np.sum((err_pmus - err_pmus_hat)**2))/np.sqrt(np.sum((err_pmus - np.average(err_pmus))**2))
        return nrmse
    
    def pearson_r(self):
        self.y_true = np.squeeze(self.y_true)
        self.y_pred = np.squeeze(self.y_pred)
        pearson_r, _ = pearsonr(self.y_true, self.y_pred)

        return pearson_r


    def evaluate(self):
        mse = mean_squared_error(self.y_true, self.y_pred)
        mae = mean_absolute_error(self.y_true, self.y_pred)
        nmse = self.normalized_mean_squared_error()
        pearson_r = self.pearson_r()
        mape = mean_absolute_percentage_error(self.y_true, self.y_pred)*100

        return mse, mae, nmse, mape, pearson_r



In [ ]:
class BlandAltman2:
    def __init__(self, output_data_unscaled, pre_data_unscaled):
        self.output_data_unscaled = output_data_unscaled
        self.pre_data_unscaled = pre_data_unscaled

    def plot(self):
       
        output_data = np.asarray(self.output_data_unscaled)
        pre_data = np.asarray(self.pre_data_unscaled)

      
        diff = (output_data - pre_data) / output_data * 100

        
        average_compliance = (output_data + pre_data) / 2

      
        mean_diff = np.mean(diff)
        std_diff = np.std(diff)

        overestimate_indices = []
        underestimate_indices = []
        # Find indices of outliers
        outlier_indices = []

        for i, percentage_error in enumerate(diff):
            if percentage_error > 20:  # overestimate
                outlier_indices.append(i)
                overestimate_indices.append(i)
            elif percentage_error < -20:  # underestimate
                outlier_indices.append(i)
                underestimate_indices.append(i)

    
        fig, ax = plt.subplots(figsize=(8, 6))
        for i in range(len(diff)):
            if i in outlier_indices:
                ax.scatter(average_compliance[i], diff[i], color='blue', s=10)
            else:
                ax.scatter(average_compliance[i], diff[i], color='blue', s=10)
        ax.axhline(mean_diff, color='gray', linestyle='--')
        ax.axhline(mean_diff + 1.96 * std_diff, color='gray', linestyle='--')
        ax.axhline(mean_diff - 1.96 * std_diff, color='gray', linestyle='--')
        ax.set_xlabel('Average Compliance Value')
        ax.set_ylabel('Percentage Difference between Predictions and Targets')
        ax.set_title('Bland-Altman Plot')
        plt.show()

        # Print indices of outliers
        print("Indices of outliers: ", outlier_indices)
        print("Indices of overestimations: ", overestimate_indices)
        print("Indices of underestimations: ", underestimate_indices)
        return outlier_indices, overestimate_indices, underestimate_indices


In [1]:
class BlandAltman1:
    def __init__(self, output_data_unscaled, pre_data_unscaled, test_indices):
        self.output_data_unscaled = output_data_unscaled
        self.pre_data_unscaled = pre_data_unscaled
        self.test_indices = test_indices
    
    def plot(self):
        
        diff = (np.asarray(self.output_data_unscaled) - np.asarray(self.pre_data_unscaled)) / np.asarray(self.output_data_unscaled) * 100

     
        mean_diff = np.mean(diff)
        std_diff = np.std(diff)
        overestimate_indices = []
        underestimate_indices = []
       
        outlier_indices = []
        
   
        for i, percentage_error in enumerate(diff):
            if percentage_error > 20:  # overestimate
                outlier_indices.append(i)
                overestimate_indices.append(i)
            elif percentage_error < -20:  # underestimate
                outlier_indices.append(i)
                underestimate_indices.append(i)

    
        fig, ax = plt.subplots(figsize=(8, 6))
        for i in range(len(diff)):
            if i in outlier_indices:
                ax.scatter(np.asarray(self.output_data_unscaled)[i], diff[i], color='red', s=10)
                ax.text(np.asarray(self.output_data_unscaled)[i], diff[i], str(self.test_indices[i]))
            else:
                ax.scatter(np.asarray(self.output_data_unscaled)[i], diff[i], color='blue', s=10)
        ax.axhline(mean_diff, color='gray', linestyle='--')
        ax.axhline(mean_diff + 1.96 * std_diff, color='gray', linestyle='--')
        ax.axhline(mean_diff - 1.96 * std_diff, color='gray', linestyle='--')
        ax.set_xlabel('Target')
        ax.set_ylabel('Percentage Difference between Predictions and Targets')
        ax.set_title('Bland-Altman Plot')
        plt.show()

        # Print indices of outliers
        print("Indices of outliers: ", [self.test_indices[i] for i in outlier_indices])
        print("Indices of overestimations: ", [self.test_indices[i] for i in overestimate_indices])
        print("Indices of underestimations: ", [self.test_indices[i] for i in underestimate_indices])
        return outlier_indices, overestimate_indices, underestimate_indices


In [ ]:
class BlandAltman:
    def __init__(self, output_data_unscaled, pre_data_unscaled, test_indices):
        self.output_data_unscaled = output_data_unscaled
        self.pre_data_unscaled = pre_data_unscaled
        self.test_indices = test_indices
    def plot(self):
       
        output_data = np.asarray(self.output_data_unscaled)
        pre_data = np.asarray(self.pre_data_unscaled)

     
        diff = (output_data - pre_data) / output_data * 100

      
        average_compliance = (output_data + pre_data) / 2

      
        mean_diff = np.mean(diff)
        std_diff = np.std(diff)

        overestimate_indices = []
        underestimate_indices = []
       
        outlier_indices = []

        for i, percentage_error in enumerate(diff):
            if percentage_error > 20:  # overestimate
                outlier_indices.append(i)
                overestimate_indices.append(i)
            elif percentage_error < -20:  # underestimate
                outlier_indices.append(i)
                underestimate_indices.append(i)

     
        fig, ax = plt.subplots(figsize=(8, 6))
        for i in range(len(diff)):
            if i in outlier_indices:
                ax.scatter(average_compliance[i], diff[i], color='red', s=10)
                ax.text(average_compliance[i], diff[i], str(self.test_indices[i]))
            else:
                ax.scatter(average_compliance[i], diff[i], color='blue', s=10)
        ax.axhline(mean_diff, color='gray', linestyle='--')
        ax.axhline(mean_diff + 1.96 * std_diff, color='gray', linestyle='--')
        ax.axhline(mean_diff - 1.96 * std_diff, color='gray', linestyle='--')
        ax.set_xlabel('Average Compliance Value')
        ax.set_ylabel('Percentage Difference between Predictions and Targets')
        ax.set_title('Bland-Altman Plot')
        plt.show()

        # Print indices of outliers
        print("Indices of outliers: ", [self.test_indices[i] for i in outlier_indices])
        print("Indices of overestimations: ", [self.test_indices[i] for i in overestimate_indices])
        print("Indices of underestimations: ", [self.test_indices[i] for i in underestimate_indices])
        return outlier_indices, overestimate_indices, underestimate_indices


In [6]:
import plotly
import pickle

In [7]:
import optuna
import random
from optuna.visualization import plot_param_importances

class Objective:
    def __init__(self, train_loader, valid_loader, device, d_model_range, nhead_range, num_layers_range, dropout_range, lr_range, n_trials, vocab_size, max_seq_len, output_dim, num_epochs, patience, weight_decay,num_hidden_layers_range,hidden_dim_range):
        self.train_loader = train_loader
        self.valid_loader = valid_loader
        self.device = device
        self.d_model_range = d_model_range
        self.nhead_range = nhead_range
        self.num_layers_range = num_layers_range
        self.dropout_range = dropout_range
        self.lr_range = lr_range
        self.n_trials = n_trials
        self.vocab_size = vocab_size
        self.max_seq_len = max_seq_len
        self.output_dim = output_dim
        self.num_epochs = num_epochs
        self.patience = patience
        self.weight_decay = weight_decay
        self.trial_num = 1
        self.best_valid_loss = float('inf')
        self.num_hidden_layers_range = num_hidden_layers_range
        self.hidden_dim_range = hidden_dim_range
        self.all_train_losses = []
        self.all_valid_losses = []
        
        self.trial_num = 1

    def save_trial_data(self, trial_num, params, model, train_losses, valid_losses):
        

        all_trials_folder_path = r'F:\junwei\final_tranformer\all_trails_(10HZ-800)_C_R'
        if not os.path.exists(all_trials_folder_path):
            os.makedirs(all_trials_folder_path)

        with open(os.path.join(all_trials_folder_path, f'trial_{trial_num}.pkl'), 'wb') as f:
            pickle.dump((model.state_dict(), params, train_losses, valid_losses), f)

      


    def __call__(self, trial):
        trial.params['d_model'] = trial.suggest_categorical('d_model', self.d_model_range)
        trial.params['nhead'] = trial.suggest_categorical('nhead', self.nhead_range)
        trial.params['weight_decay'] = trial.suggest_loguniform("weight_decay", *self.weight_decay)
        trial.params['num_layers'] = trial.suggest_int("num_layers", *self.num_layers_range)
        trial.params['dropout'] = trial.suggest_float("dropout", *self.dropout_range)
        trial.params['lr'] = trial.suggest_loguniform("lr", *self.lr_range)
        num_hidden_layers = trial.suggest_int("num_hidden_layers", *self.num_hidden_layers_range)
        hidden_layers = []
        for i in range(num_hidden_layers):
            hidden_dim = trial.suggest_categorical(f"hidden_dim_{i}", self.hidden_dim_range)
            hidden_layers.append(hidden_dim)


        model = Transformer(self.vocab_size, self.max_seq_len, trial.params['d_model'], trial.params['nhead'], trial.params['num_layers'], self.output_dim, self.device, trial.params['dropout'], hidden_layers).to(self.device)
        mse_loss = nn.L1Loss()
        optimizer = optim.RMSprop(model.parameters(), lr=trial.params['lr'], weight_decay=trial.params['weight_decay'])

        trainer = Trainer(model=model, 
                          train_loader=self.train_loader, 
                          valid_loader=self.valid_loader, 
                          num_epochs=self.num_epochs, 
                          patience=self.patience, 
                          device=self.device, 
                          mse_loss=mse_loss, 
                          optimizer=optimizer
                          )

        train_losses, valid_losses, _ = trainer.train()
        self.all_train_losses.append(train_losses)
        self.all_valid_losses.append(valid_losses)

        params = trial.params
        params['d_model'] = trial.params['d_model']
        params['nhead'] = trial.params['nhead']
        params['hidden_layers'] = [trial.params[f'hidden_dim_{i}'] for i in range(trial.params['num_hidden_layers'])]

        self.save_trial_data(self.trial_num, params, model, train_losses, valid_losses)
        


        if valid_losses[-1] < self.best_valid_loss:
            self.best_valid_loss = valid_losses[-1]
            self.best_model = model
            self.best_train_losses = train_losses
            self.best_valid_losses = valid_losses

        self.trial_num += 1
        return valid_losses[-1]  # Return the last validation loss
    
    def plot_losses(self):
        for i in range(self.n_trials):
            plt.plot(self.all_train_losses[i], label='Train Loss')
            plt.plot(self.all_valid_losses[i], label='Validation Loss')
            plt.title(f'Trial {i+1}')
            plt.xlabel('Epoch')
            plt.ylabel('Loss')
            plt.legend()
            plt.show()

    def optimize(self):
        sampler = optuna.samplers.TPESampler(seed=37)
        study = optuna.create_study(sampler = sampler, direction="minimize")
        study.optimize(self, n_trials=self.n_trials, n_jobs=1)
        best_trial = study.best_trial

        best_params = best_trial.params
        best_params['d_model'] = best_trial.params['d_model']
        best_params['nhead'] = best_trial.params['nhead']
        best_params['hidden_layers'] = [best_trial.params[f'hidden_dim_{i}'] for i in range(best_trial.params['num_hidden_layers'])]

        folder_path = r'F:\junwei\final_tranformer\best_model_C_R(10HZ-800)'
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

    

        with open(os.path.join(folder_path, 'best_model_and_params.pkl'), 'wb') as f:
            pickle.dump((self.best_model.state_dict(), best_params, self.best_train_losses, self.best_valid_losses), f)

            
        fig = plot_param_importances(study)
        fig.show()

        plt.plot(self.best_train_losses, label='Training Loss')
        plt.plot(self.best_valid_losses, label='Validation Loss')
        plt.legend()
        plt.show()

        return (best_trial.value, best_params)


In [8]:
class BestModelLoader:
    def __init__(self, path, vocab_size, max_seq_len, output_dim, device):
        self.path = path
        self.vocab_size = vocab_size
        self.max_seq_len = max_seq_len
        self.output_dim = output_dim
        self.device = device

    def load_best_model(self):
   
        with open(self.path, 'rb') as f:
            best_model_state_dict, best_params, best_train_losses, best_valid_losses = pickle.load(f)


     
        model = Transformer(self.vocab_size, self.max_seq_len, best_params['d_model'], best_params['nhead'], best_params['num_layers'], self.output_dim, self.device, best_params['dropout'], best_params['hidden_layers']).to(self.device)

     
        best_model_state_dict = {k: v.to(self.device) for k, v in best_model_state_dict.items()}

  
        model.load_state_dict(best_model_state_dict)

        return model,best_train_losses, best_valid_losses
    
    def plot_loss(self, best_train_losses, best_valid_losses):
        # Plot the train and validation losses
        plt.plot(best_train_losses, label='Training Loss')
        plt.plot(best_valid_losses, label='Validation Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

    def test_model(self, model, test_loader):
        model.eval()
        num_samples = 0
        preds = []
        targets_list = []

        with torch.no_grad():
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                # Forward pass
                outputs = model(inputs)
                outputs = outputs.cpu().numpy()
                targets = targets.cpu().numpy()

                num_samples += len(targets)
                preds.append(outputs)
                targets_list.append(targets)

        preds = np.concatenate(preds, axis=0)
        targets = np.concatenate(targets_list, axis=0)
        
        return preds, targets

In [9]:
def normalize_data1(data):
    minimum = np.min(data, axis=(0, 1), keepdims=True)
    maximum = np.max(data, axis=(0, 1), keepdims=True)
    data_norm = (data - minimum) / (maximum - minimum)
    return minimum, maximum, data_norm

def normalize_data2(data, minimum = None,maximum = None):
    if minimum is None:
        minimum = np.min(np.min(data))
    if maximum is None:
        maximum = np.max(np.max(data))
    data_norm = (data - minimum) / (maximum - minimum)
    return minimum, maximum, data_norm

def denormalize_data(data, minimum, maximum):
    return data * (maximum - minimum) + minimum


In [3]:
def scatter(true,pre,name):   
    plt.figure(figsize=(8, 6))
    plt.scatter(true, pre, c='blue',s=10)

    lims = [
        np.min([plt.xlim(), plt.ylim()]),  # min of both axes
        np.max([plt.xlim(), plt.ylim()]),  # max of both axes
    ]
    plt.plot(lims, lims, 'k-', alpha=0.75, zorder=0)
    plt.plot(lims, [lims[0]*0.8, lims[1]*0.8], 'r-', alpha=0.5)  
    plt.plot(lims, [lims[0]*1.2, lims[1]*1.2], 'r-', alpha=0.5)  

    plt.xlabel('True Values')
    plt.ylabel('Predicted Values')
    plt.title(name + ' True vs Predicted Values')
    plt.grid(True)
    plt.show()

In [ ]:
def find_ids(index_list, test_data1, dataset1):
    id_list = []
    for i in index_list:
        if i in test_data1.index:
            id_value = test_data1.loc[i, 'ID']
            id_list.append(id_value)
    dataset2 = dataset1[dataset1['ID'].isin(id_list)]
    return dataset2 , id_list 